In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import time

In [2]:
import os

In [3]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


In [4]:
root_address = 'https://classroom.udacity.com/nanodegrees/nd101/dashboard/overview'
# root_address = 'https://www.udacity.com'

In [5]:
#setup option for chrome profile
chrome_options = Options()
# chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")

#start web driver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.implicitly_wait(60)

driver.get(root_address)

wait = WebDriverWait(driver, 30)

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103


[WDM] - Driver [/Users/huohsien/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [6]:
email = driver.find_element_by_xpath("//input[@data-testid='signin-email']")
email

<selenium.webdriver.remote.webelement.WebElement (session="bca557318a76f85873c860e48f16de48", element="cca2eb32-fc9e-4e50-a086-ab3ac4c9a148")>

In [7]:
def sync_get_element_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element_by_xpath(xpath)

In [8]:
def sync_get_elements_by_xpath(xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_elements_by_xpath(xpath)

In [9]:
def sign_in():
    email = sync_get_element_by_xpath("//input[@data-testid='signin-email']")
    password = sync_get_element_by_xpath("//input[@data-testid='signin-password']")

    email.send_keys("huohsien@gmail.com")
    password.send_keys("jj1216")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb")))
    sign_in_btn = driver.find_element_by_class_name("button_primary__1qhjh.button__btn__2MHEg.form_primary-button__37eaW.button_standard__1p9sb") 
    sign_in_btn.click()

In [10]:
def check_lesson_complete(item)->bool:
    e1 = item.find_element_by_xpath("./div/div")
    c = e1.get_attribute('class')
    if c == 'index--lesson-card--mwX1V index--card--3DZMr shared--card--3X88h':
        return True
    else:
        return False

In [11]:
def expand_all_complete_lessons():
    #click open the ones that were marked Complete
    lessons = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    for lesson in lessons:
        if check_lesson_complete(lesson):
            lesson.click()
    element = driver.find_element_by_xpath("//*[h3='lesson 1']")
    driver.execute_script("return arguments[0].scrollIntoView(true);", element)
#     time.sleep(1)

In [12]:
if check_exists_by_xpath("//div[div='Sign in to your account']"):
    sign_in()

In [13]:
root_folder = './dlnd_course_downloaded'

In [14]:
os.makedirs(root_folder, exist_ok=True)

In [15]:
# Get topic titles
topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
num_topics = len(topic_elms)

for idx_topics in range(num_topics):
    # iterate each topic
    #
    topic_name = ''.join(topic_elms[idx_topics].text.split('\n')[0:2])                   
    os.makedirs(os.path.join(root_folder, topic_name), exist_ok=True)
    print("Topic: ", topic_name)
    link_topic = topic_elms[idx_topics].find_element_by_xpath("./a")
    link_topic.click()     # click into a topic
    # In the lesson list page
    #
    expand_all_complete_lessons()
    
    #iterate each lesson
    #
    lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")
    num_lessons = len(lesson_elms)
    for idx_lesson in range(num_lessons):
        # iterate each lesson
        #
        lesson_elm = lesson_elms[idx_lesson]
        lesson_name = ' - '.join(lesson_elm.text.split('\n')[1:3])                   
        os.makedirs(os.path.join(root_folder, topic_name,lesson_name), exist_ok=True)
        print(lesson_name)
                
        link_lesson = lesson_elm.find_element_by_xpath("//*[span='View Lesson']")
        link_lesson.click()  # click into a lesson
        
        # In the content page of a lesson. loop through all parts in it
        part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")
        num_parts = len(part_elms)
        for idx_part in range(num_parts):
            # iterate each part
            part_elm = part_elms[idx_part]
            part_name = part_elm.text
            os.makedirs(os.path.join(root_folder, topic_name,lesson_name, part_name), exist_ok=True)
            print("part_name= ", part_name)
            file_name = os.path.join(root_folder, topic_name,lesson_name, part_name, "content.html")
            link_part = part_elm.find_element_by_xpath("./a")
            link_part.click()     # click into a part
            time.sleep(1)
            main_layout_contents = sync_get_elements_by_xpath("//div[@id='main-layout-content']/..")
            assert len(main_layout_contents) == 1
            
            main_layout_content_inner_html = main_layout_contents[0].get_attribute('innerHTML')
            with open(file_name, 'w') as f:
                f.write(main_layout_content_inner_html)
#             print("content= ", content)
#             try:
#                 print("iframe XXX = ", content.find_element_by_xpath("//iframe").get_attribute('id'))
#                 iframe = content.find_element_by_xpath("//iframe")
#                 print("src= ", iframe.get_attribute('src'))
#             except NoSuchElementException:
#                 print("no video in this part")
            
            
            part_elms = sync_get_elements_by_xpath("//*[@id='tree-concepts']/ol/*")
        for i in range(num_parts):
            driver.back()
        expand_all_complete_lessons()
        time.sleep(1)
        lesson_elms = sync_get_elements_by_xpath("//div[@data-test='part-syllabus']/ol/li")

    driver.back()
    topic_elms = sync_get_elements_by_xpath("//*[@id='tree-core-curriculum']/*")
    

Topic:  1. Introduction to Deep Learning
LESSON 1 - Welcome to the Deep Learning Nanodegree Program
part_name=  1. Welcome to the Deep Learning Nanodegree Program
part_name=  2. Meet Your Instructors
part_name=  3. Program Structure
part_name=  4. Community Guidelines
part_name=  5. Prerequisites
part_name=  6. Getting Set Up
LESSON 2 - Nanodegree Career Services
part_name=  1. Welcome to the Deep Learning Nanodegree Program
part_name=  2. Meet Your Instructors
part_name=  3. Program Structure
part_name=  4. Community Guidelines
part_name=  5. Prerequisites
part_name=  6. Getting Set Up
LESSON 3 - Knowledge, Community, and Careers
part_name=  1. Welcome to the Deep Learning Nanodegree Program
part_name=  2. Meet Your Instructors
part_name=  3. Program Structure
part_name=  4. Community Guidelines
part_name=  5. Prerequisites
part_name=  6. Getting Set Up
LESSON 4 - Get Help with Your Account
part_name=  1. Welcome to the Deep Learning Nanodegree Program
part_name=  2. Meet Your Instruc

TimeoutException: Message: 


In [ ]:
# link_topic.click() 

In [ ]:
# expand_all_complete_lessons()



In [ ]:
# element = driver.find_element_by_xpath("//*[h3='lesson 1']")
# driver.execute_script("return arguments[0].scrollIntoView(true);", element)